In [1]:
import imageio
from imageio.core.util import asarray as imgToArr
import matplotlib.pylab as pylab
import numpy as np
import pandas as pd

In [2]:
# Columns: Frame, Brake, GazeX, GazeY
dataFile = './data/cleaned_data.csv'
df = pd.read_csv(dataFile, delimiter='\t')

brake = df[df['Brake'] > 0]
nonbrake = df[df['Brake'] == 0]
nonbrake = nonbrake[:len(brake)]  # Braking is far fewer than nonbraking, so trim down
df = pd.concat([brake, nonbrake])
df = df.drop(df[df['GazeX'] < 0].index)
df = df.drop(df[df['GazeY'] < 0].index)
df = df.dropna()
df = df.reset_index(drop=True)  # Resets the index to the usual 0, 1, 2, ...

In [3]:
def get_glimpse(image, x, y, stride=14):
    """Returns a subsection (glimpse) of the image centered on the given point."""
    x = int(x)  # Force to int
    y = int(y)  # Force to int
    min_x = x - stride
    max_x = x + stride
    
    min_y = y - stride
    max_y = y + stride
    image_glimpse = image[min_y:max_y, min_x:max_x, :]  # NOTE: row, column, RGB
#     image_glimpse = image[min_y:max_y, min_x:max_x, 0]  # NOTE: row, column, RGB; everything is greyscale; flatten RGB layer
    return imgToArr(image_glimpse)

In [4]:
filename = 'data/driving.avi'
vid = imageio.get_reader(filename,  'ffmpeg')
batch = 1
count = 1
frames = np.zeros((10000, 28, 28, 3))
gazes = np.zeros((10000, 2))
braking = np.zeros((10000, 2))
for i, row in df.iterrows():
    frame = row['Frame']
    x = row['GazeX']
    y = row['GazeY']
    brake = row['Brake']
    
    image = vid.get_data(frame)
    glimpse = get_glimpse(image, x, y)
    if glimpse.shape != (28, 28, 3):
        continue
    frames[count] = glimpse
    gazes[count] = np.array([x, y])
    if brake == 0:
        braking[count] = np.array([1, 0])  # nonbraking on left
    else:
        braking[count] = np.array([0, 1])  # braking on right
    count += 1
    if count % 10000 == 0:
        print("processed: {0}".format(count))

        frames = np.array(frames)
        print(frames.shape)
        print(gazes.shape)
        print(braking.shape)
        SAVE_FILE_NAME = 'data/glimpse_batchc_{0}'.format(batch)
        np.savez_compressed(SAVE_FILE_NAME, frames=frames, gazes=gazes, braking=braking)
        print("Saved " + SAVE_FILE_NAME)

        # Reset
        batch += 1
        count = 1
        frames = np.zeros((10000, 28, 28, 3))
        gazes = np.zeros((10000, 2))

processed: 10000
(10000, 28, 28, 3)
(10000, 2)
(10000, 2)
Saved data/glimpse_batchc_1
processed: 10000
(10000, 28, 28, 3)
(10000, 2)
(10000, 2)
Saved data/glimpse_batchc_2
processed: 10000
(10000, 28, 28, 3)
(10000, 2)
(10000, 2)
Saved data/glimpse_batchc_3
processed: 10000
(10000, 28, 28, 3)
(10000, 2)
(10000, 2)
Saved data/glimpse_batchc_4
processed: 10000
(10000, 28, 28, 3)
(10000, 2)
(10000, 2)
Saved data/glimpse_batchc_5
processed: 10000
(10000, 28, 28, 3)
(10000, 2)
(10000, 2)
Saved data/glimpse_batchc_6
processed: 10000
(10000, 28, 28, 3)
(10000, 2)
(10000, 2)
Saved data/glimpse_batchc_7
processed: 10000
(10000, 28, 28, 3)
(10000, 2)
(10000, 2)
Saved data/glimpse_batchc_8
